In [1]:
%matplotlib inline

import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

from sklearn.model_selection import train_test_split


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(999)
np.random.seed(0)

In [2]:
channels = 2
filters = 64

n_epocs = 20
batch_size = 511
test_size = 0.2
learning_rate = 0.002 * (batch_size / 1024.0)
momentum = 0.0

In [3]:
class Connect4Dataset(data.Dataset):
    def __init__(self, boards, values):
        assert len(boards) == len(values)
        self.boards = boards
        self.values = values
        
    def __len__(self):
        return len(self.boards)
    
    def __getitem__(self, idx):
        return self.boards[idx], self.values[idx]

boards = torch.load('/home/richard/Downloads/connect4_boards.pth').numpy()
values = torch.load('/home/richard/Downloads/connect4_values.pth').numpy()

# Here we don't want to have the player to move channel
boards = boards[:, 3 - channels:]

board_train, board_test, value_train, value_test = train_test_split(boards, values, test_size=test_size, shuffle=True)

In [4]:
# # augment data
v_wins = value_train[value_train == 1.0]
v_draws = value_train[value_train == 0.5]
v_losses = value_train[value_train == 0.0]
b_wins = board_train[value_train == 1.0]
b_draws = board_train[value_train == 0.5]
b_losses = board_train[value_train == 0.0]

n_wins = len(v_wins)
n_draws = len(v_draws)
n_losses = len(v_losses)

v_augmented_draws = np.repeat(v_draws, n_wins/n_draws)
v_augmented_losses = np.repeat(v_losses, n_wins/n_losses)
b_augmented_draws = np.repeat(b_draws, n_wins/n_draws, axis=0)
b_augmented_losses = np.repeat(b_losses, n_wins/n_losses, axis=0)

extra_draw_idx = np.random.choice(range(len(v_draws)), n_wins - len(v_augmented_draws), replace=False)
extra_losses_idx = np.random.choice(range(len(v_losses)), n_wins - len(v_augmented_losses), replace=False)

v_augmented_draws = np.hstack([v_augmented_draws, v_draws[extra_draw_idx]])
v_augmented_losses = np.hstack([v_augmented_losses, v_losses[extra_losses_idx]])
b_augmented_draws = np.concatenate([b_augmented_draws, b_draws[extra_draw_idx]], axis=0)
b_augmented_losses = np.concatenate([b_augmented_losses, b_losses[extra_losses_idx]], axis=0)

value_train = np.hstack([v_wins, v_augmented_draws, v_augmented_losses])
board_train = np.concatenate([b_wins, b_augmented_draws, b_augmented_losses], axis=0)

train = Connect4Dataset(torch.from_numpy(board_train), torch.from_numpy(value_train))
test = Connect4Dataset(torch.from_numpy(board_test), torch.from_numpy(value_test))

train_gen = data.DataLoader(train, batch_size, shuffle=True)
test_gen = data.DataLoader(test, batch_size, shuffle=True)

In [ ]:
from src.connect4.neural import value_net as net

def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.0000001)
    elif type(m) == nn.Conv2d:
        nn.init.normal_(m.weight, std=0.0000001)
    elif type(m) == nn.BatchNorm2d:
        nn.init.normal_(m.weight, std=0.0000001)
#         nn.init.constant_(m.bias, 0)

# net.apply(init_normal)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

In [ ]:
# criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = optim.Adam(net.parameters())

In [ ]:
def categorise_predictions(preds):
    preds = preds * 3.0
    torch.floor_(preds)
    preds = preds / 2.0
    return preds

class Stats():
    def __init__(self):
        self.n = 0
        self.average_value = 0.0
        self.total_loss = 0.0
        self.smallest = 1.0
        self.largest = 0.0
        self.correct = {i : 0 for i in [0.0, 0.5, 1.0]}
        self.total = {i : 0 for i in [0.0, 0.5, 1.0]}
    
    @property
    def loss(self):
        return self.total_loss / self.n
    
    @property
    def accuracy(self):
        return float(sum(self.correct.values())) / self.n
        
    def __repr__(self):
        x = "Average loss:  " + "{:.5f}".format(self.loss) + \
            ",  Accuracy:  " + "{:.5f}".format(self.accuracy) + \
            ",  Smallest:  " + "{:.5f}".format(self.smallest) + \
            ",  Largest:  " + "{:.5f}".format(self.largest) + \
            ",  Average:  " + "{:.5f}".format(self.average_value / self.n)
        
        for k in self.correct:
            x += "\nCategory, # Members, # Correct Predictions:  {}, {}, {}".format(
                k,
                self.total[k],
                self.correct[k])
        return x
    
    def update(self, outputs, values, loss):
        self.n += len(values)
        self.average_value += outputs.sum().item()
        self.total_loss += loss.item() * len(values)
        self.smallest = min(self.smallest, torch.min(outputs).item())
        self.largest = max(self.largest, torch.max(outputs).item())

        categories = categorise_predictions(outputs)
        values = values.view(-1)
        categories = categories.view(-1)

        for k in self.correct:
            idx = (values == k).nonzero()
            self.total[k] += len(idx)
            self.correct[k] += len(torch.eq(categories[idx], values[idx]).nonzero()) 

In [ ]:
%%time

from laplotter import LossAccPlotter

plotter = LossAccPlotter()

for epoch in range(n_epocs):
    
    net = net.train()
    train_stats = Stats()
    test_stats = Stats()
    
    for board, value in train_gen:
        board, value = board.to(device), value.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = net(board)
        loss = criterion(output, value)
        loss.backward()
        optimizer.step()
        
        train_stats.update(output, value, loss)

    # validate
    with torch.set_grad_enabled(False):
        net = net.eval()
        for board, value in test_gen:
            board, value = board.to(device), value.to(device)

            output = net(board)
            loss = criterion(output, value)
            
            test_stats.update(output, value, loss)
            
    print("Epoch:  ", epoch, "  Train:\n", train_stats, "\nTest:\n", test_stats)
    plotter.add_values(epoch,
                       loss_train=train_stats.loss, acc_train=train_stats.accuracy,
                       loss_val=test_stats.loss, acc_val=test_stats.accuracy)
            

print('Finished Training')

Epoch:   2   Train:
 Average loss:  0.19129,  Accuracy:  0.48539,  Smallest:  0.00000,  Largest:  1.00000,  Average:  0.49355
Category, # Predictions, # Correct:  0.0, 35539, 22758
Category, # Predictions, # Correct:  0.5, 35539, 23891
Category, # Predictions, # Correct:  1.0, 35539, 5102 
Test:
 Average loss:  0.17624,  Accuracy:  0.27805,  Smallest:  0.00000,  Largest:  1.00000,  Average:  0.67491
Category, # Predictions, # Correct:  0.0, 3300, 1932
Category, # Predictions, # Correct:  0.5, 1278, 786
Category, # Predictions, # Correct:  1.0, 8934, 1039
((8934-1039)*0.33 + (3300 - 1932) * 0.33 + (1278 - 786) * 0.16) / (3300+1278+8934) = 0.23
So explain to me how the test set had an average loss of 0.18!?
Still large for epoch 3
((8934-2201)*0.33 + (3300 - 2108) * 0.33 + (1278 - 909) * 0.16) / (3300+1278+8934) = 0.20

In [ ]:
plotter.fig

In [ ]:
# validate
with torch.set_grad_enabled(False):
    net = net.eval()
    correct = {i : 0 for i in [0, 0.5, 1]}
    total = {i : 0 for i in [0, 0.5, 1]}
    for board, value in test_gen:
        board, value = board.to(device), value.to(device)

        outputs = net(board)
        categories = categorise_predictions(outputs)

        for k in correct:
            idx = (categories == k).nonzero()
            total[k] += len(idx)
            correct[k] += (categories[idx] == value[idx]).nonzero().sum().item()
    for k in correct:
        print('Category, # Predictions, Accuracy of the network on the test%s: %d %d %%' % (
            k,
            total[k],
            (100 * float(correct[k]) / float(total[k])) if total[k] != 0 else 0.))

In [ ]:
assert(False)
# save that crap
torch.save({
    'net_state_dict': net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss},
    '/home/richard/Downloads/nn.pth')

In [ ]:
# alternatively load it
assert(False)
checkpoint = torch.load('/home/richard/Downloads/nn.pth')
net.load_state_dict(checkpoint['net_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss = checkpoint['loss']